# Asteroid Classification
This notebook contains a jupyter notebook to train and test a dense neural network for classifying asteroids as hazardous or not. Because of data imbalance, in testing the network only achieved about 75% binary accuracy, however a much greater percentage of these were false positive than negatives.

In [1]:
import tensorflow as tf 
from tensorflow import keras 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import io
import os
import tempfile

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

relative_velocity_flag = False
miss_distance_flag = False

### Data Preprocessing

In [2]:
csv_link = 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/celesital-bodies/neo_v2.csv'
csv_file = requests.get(csv_link).content
dataset = pd.read_csv(io.StringIO(csv_file.decode('utf-8')))

In [3]:
change_to_float = {column:float for column, dtype in zip(dataset.columns.tolist(), dataset.dtypes.tolist()) if dtype == np.bool_}
dataset = dataset.astype(change_to_float)
dataset.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,0.0,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,0.0,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,0.0,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,0.0,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,0.0,20.09,1.0


In [4]:
neg, pos = np.bincount(dataset['hazardous'])
total = neg + pos
print(f"Percent positive is {(pos / neg) * 100} %")

Percent positive is 10.781013707985755 %


In [5]:
try:
    dataset_processed = dataset.drop(['sentry_object','orbiting_body'], axis=1)
except (AttributeError, KeyError):
    print("already removed")

In [6]:
dataset_processed.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,20.09,1.0


In [7]:
def scale_down(dataset=None, column=None):
    initial_range = np.amax(dataset[column]) - np.amin(dataset[column])
    eps = 0.001
    dataset[column] = np.log(dataset[column] + eps)
    end_range = np.amax(dataset[column]) - np.amin(dataset[column])

    return initial_range, end_range

In [8]:
if not relative_velocity_flag:
    initial_range, end_range = scale_down(column='miss_distance', dataset=dataset_processed)
    relative_velocity_flag = True
else:
    print("scaling already complete")

print(f'Initial_range: {initial_range}\nend_range: {end_range}')

if not miss_distance_flag:
    initial_range, end_range = scale_down(column='relative_velocity', dataset=dataset_processed)
    miss_distance_flag = True
else:
    print("scaling already complete")

print(f'Initial_range: {initial_range}\nend_range: {end_range}')

Initial_range: 74791905.91968124
end_range: 9.313674550575696
Initial_range: 236786.78165533658
end_range: 7.060857763424829


In [30]:
scaler = StandardScaler()
train_data, test_data = train_test_split(dataset_processed.drop(['id', 'name'], axis=1), test_size=0.1)
_, regurgitate_data = train_test_split(dataset, test_size=0.1)
train_data, validation_data = train_test_split(train_data, test_size=0.1)

train_labels, test_labels, validation_labels = (np.array(train_data.pop('hazardous')),
                                                np.array(test_data.pop('hazardous')), 
                                                np.array(validation_data.pop('hazardous')))
bool_train_labels = train_labels != 0
train_features, test_features, validation_features = (np.clip(scaler.fit_transform(np.array(train_data)), 0, 60),
                                                      np.clip(scaler.fit_transform(np.array(test_data)), 0, 60), 
                                                      np.clip(scaler.fit_transform(np.array(validation_data)), 0, 60))

### Model Creation

In [10]:
model_metrics = [keras.metrics.BinaryAccuracy(),
                       keras.metrics.Precision(),
                       keras.metrics.Recall(),
                       keras.metrics.TruePositives(),
                       keras.metrics.TrueNegatives(),
                       keras.metrics.FalseNegatives(),
                       keras.metrics.FalsePositives(),
                       keras.metrics.AUC(name='auc'),
                       keras.metrics.AUC(name='prc', curve='PR')
                       ]

In [11]:
def make_model(metrics=model_metrics, output_bias=None):
    if output_bias:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = tf.keras.Sequential([
        keras.layers.Dense(5, activation='relu', input_shape=(train_features.shape[-1],), 
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(5, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(5, activation='relu',
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(3, activation='relu',
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )
    
    return model

In [12]:
# epochs, batch size, and early stopping
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True
)

# Setting the initial bias
initial_bias = np.log([pos/neg])
model = make_model(output_bias=initial_bias)
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [13]:
# classification weights
weight_for_0 = (1 / neg) * (total / 2)
weight_for_1 = (1 / pos) * (total / 2)

class_weights = {0: weight_for_0, 1: weight_for_1}

print(f'Non-Hazardous Weight: {weight_for_0}\nHazardous Weight: {weight_for_1}')

Non-Hazardous Weight: 0.5539050685399288
Hazardous Weight: 5.137782805429865


In [14]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)
weighted_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 30        
                                                                 
 dropout_3 (Dropout)         (None, 5)                 0         
                                                                 
 dense_7 (Dense)             (None, 5)                 30        
                                                                 
 dense_8 (Dense)             (None, 5)                 30        
                                                                 
 dropout_4 (Dropout)         (None, 5)                 0         
                                                                 
 dense_9 (Dense)             (None, 3)                 18        
                                                                 
 dense_10 (Dense)            (None, 3)                

In [15]:
weighted_model.fit(train_features, 
                    train_labels, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=[early_stopping], 
                    validation_data=(validation_features, validation_labels), 
                    class_weight=class_weights)

Epoch 1/100
36/36 [==============================] - 5s 49ms/step - loss: 1.1039 - binary_accuracy: 0.8794 - precision: 0.0468 - recall: 0.0123 - true_positives: 88.0000 - true_negatives: 64613.0000 - false_negatives: 7082.0000 - false_positives: 1793.0000 - auc: 0.4141 - prc: 0.0783 - val_loss: 0.3827 - val_binary_accuracy: 0.9011 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives: 7367.0000 - val_false_negatives: 797.0000 - val_false_positives: 12.0000 - val_auc: 0.3405 - val_prc: 0.0737
Epoch 2/100
36/36 [==============================] - 0s 7ms/step - loss: 0.9124 - binary_accuracy: 0.8139 - precision: 0.0972 - recall: 0.1098 - true_positives: 787.0000 - true_negatives: 59096.0000 - false_negatives: 6383.0000 - false_positives: 7310.0000 - auc: 0.4947 - prc: 0.0967 - val_loss: 0.4409 - val_binary_accuracy: 0.8697 - val_precision: 0.1825 - val_recall: 0.0966 - val_true_positives: 77.0000 - val_true_negatives: 7034.0000 - val_fa

In [16]:
weighted_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=1)

5/5 [==============================] - 0s 4ms/step - loss: 0.4808 - binary_accuracy: 0.7450 - precision: 0.2570 - recall: 0.8740 - true_positives: 763.0000 - true_negatives: 6005.0000 - false_negatives: 110.0000 - false_positives: 2206.0000 - auc: 0.8674 - prc: 0.3221


[0.4807872176170349,
 0.7450462579727173,
 0.25698888301849365,
 0.873997688293457,
 763.0,
 6005.0,
 110.0,
 2206.0,
 0.8674417734146118,
 0.3220868706703186]

In [17]:
results = weighted_model.predict(test_features)
results[results<0.5]=0
results[results>0.5]=1

284/284 [==============================] - 1s 2ms/step


In [18]:
len(results)

9084

In [34]:
regurgitate_data['predicted_values'] = results

In [33]:
def prediction_classification(input_set):
    if all(input_set):
        return "True_Positive"
    elif not input_set[0] and input_set[1]:
        return "False_Positive"
    elif not input_set[0] and not input_set[1]:
        return "True_Negative"
    else:
        return "False_Negative"

In [32]:
def boolean_transformer(input_set, positive_identifier, negative_identifier):
    input_set[input_set == 0] = negative_identifier
    input_set[input_set == 1] = positive_identifier

    return input_set 

In [35]:
regurgitate_data['Classification_Success'] = \
list(map(prediction_classification, list(zip(regurgitate_data['hazardous'], regurgitate_data['predicted_values']))))

In [36]:
regurgitate_data['hazardous'] = boolean_transformer(regurgitate_data['hazardous'], 'hazardous', 'not_hazardous')
regurgitate_data['predicted_values'] = boolean_transformer(regurgitate_data['predicted_values'], 'predicted_hazardous', 'predicted_not_hazardous')

C:\Users\ahuma\AppData\Local\Temp/ipykernel_16504/329045822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_set[input_set == 0] = negative_identifier
C:\Users\ahuma\AppData\Local\Temp/ipykernel_16504/329045822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_set[input_set == 1] = positive_identifier


In [37]:
regurgitate_data.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous,predicted_values,Classification_Success
31028,3523943,(2010 KO10),0.010105,0.022596,42983.753068,1.732615e+05,Earth,0.0,27.10,not_hazardous,predicted_not_hazardous,True_Negative
37766,2096590,96590 (1998 XB),1.434019,3.206564,71153.282726,5.511275e+07,Earth,0.0,16.34,not_hazardous,predicted_hazardous,False_Positive
46124,54190790,(2021 QK2),0.035201,0.078712,53485.543575,6.035976e+07,Earth,0.0,24.39,not_hazardous,predicted_not_hazardous,True_Negative
1586,3767315,(2017 BY32),0.014607,0.032662,65516.623343,7.131711e+07,Earth,0.0,26.30,not_hazardous,predicted_not_hazardous,True_Negative
69274,3732907,(2015 UW67),0.073207,0.163697,31041.810196,7.506213e+06,Earth,0.0,22.80,not_hazardous,predicted_not_hazardous,True_Negative


In [39]:
regurgitate_data.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous,predicted_values,Classification_Success
31028,3523943,(2010 KO10),0.010105,0.022596,42983.753068,1.732615e+05,Earth,0.0,27.10,not_hazardous,predicted_not_hazardous,True_Negative
37766,2096590,96590 (1998 XB),1.434019,3.206564,71153.282726,5.511275e+07,Earth,0.0,16.34,not_hazardous,predicted_hazardous,False_Positive
46124,54190790,(2021 QK2),0.035201,0.078712,53485.543575,6.035976e+07,Earth,0.0,24.39,not_hazardous,predicted_not_hazardous,True_Negative
1586,3767315,(2017 BY32),0.014607,0.032662,65516.623343,7.131711e+07,Earth,0.0,26.30,not_hazardous,predicted_not_hazardous,True_Negative
69274,3732907,(2015 UW67),0.073207,0.163697,31041.810196,7.506213e+06,Earth,0.0,22.80,not_hazardous,predicted_not_hazardous,True_Negative


In [27]:
regurgitate_data.to_csv(path_or_buf=r'C:\Users\ahuma\Desktop\python projects\REHS2022\Final-Project\ExtraKGs\celesital-bodies\processed_data.csv', index=False)

In [ ]:
#keras.backend.clear_session()